In [ ]:
#test code
# Generate images from noise, using the generator network.
f, a = plt.subplots(4, 10, figsize=(10, 4))
for i in range(10):
    # Noise input.
    z = np.random.uniform(-1., 1., size=[4, noise_dim])
    g = sess.run(gen_sample, feed_dict={noise_input: z})
    for j in range(4):
        # Generate image from noise. Extend to 3 channels for matplot figure.
        img = np.reshape(np.repeat(g[j][:, :, np.newaxis], 3, axis=2),
                         newshape=(28, 28, 3))
        a[j][i].imshow(img)

In [ ]:
#test code
if step == 0 or (step + 1) % 10 == 0:
    generated_samples = model.sample_generator(noise_test, batch_size=visualization_num)
    savepath = output_dir + '/output_' + 'EP' + str(epoch).zfill(3) + "_Batch" + str(step).zfill(6) + '.jpg'
    save_visualization(generated_samples, (14, 14), save_path=savepath)